In [ ]:
## Import necessary modules
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve, matthews_corrcoef
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import seaborn as sns
import pandas as pd
import csv
import math
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score,KFold,StratifiedKFold,cross_val_predict
from sklearn.metrics import classification_report
import sklearn
import tensorflow as tf
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, LayerNormalization, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

In [ ]:
#!pip install --upgrade scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Clathrin/weightedBTGA_Clathrin_Training.csv', header=None)

In [ ]:
df.shape

(2421, 970)

In [ ]:
from sklearn.preprocessing import StandardScaler
# Initialise the Scaler
#scaler = StandardScaler()

# To scale data
#scaler.fit(df)
#from sklearn.preprocessing import StandardScaler
# Initialise the Scaler
#scaler = StandardScaler()

StandardScaler()

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(999, inplace=True)

In [ ]:
# Split the dataset into features (X) and labels (y)
X = df.iloc[:, 0:969].values
y = df.iloc [:, 969].values
print(X.shape)
print(y.shape)

(2421, 969)
(2421,)


In [ ]:
# Split into training and test set
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#             X, y, test_size = 0.2, random_state=42)
# print (len(X_train),len(X_test),len(y_train),len(y_test))
# X, y = make_classification(n_samples=2421, n_classes=2, random_state=42)
# X_train = np.expand_dims(X_train, axis=1)
# print(X_train.shape)

1936 485 1936 485
(1936, 1, 969)


In [ ]:
# Reshape the features to a 3D array for 1D CNN
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
print(X.shape)

# Normalize the features
#X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

(2421, 1, 969)


In [ ]:
X_train = X;
y_train = y;

config = {
    'dropout_rate': 0.2,
    'activation': 'tanh',
    'recurrent_dropout': 0.0,
    'kernel_regularizer': l2(0.0001),
}

def get_model(config):
  model = tf.keras.Sequential([
    Bidirectional(LSTM(128, return_sequences=True, activation=config['activation'], kernel_regularizer=config['kernel_regularizer'], recurrent_dropout=config['recurrent_dropout']), input_shape=(1, X_train.shape[2])),
    Dropout(config['dropout_rate']),
    BatchNormalization(),
    Bidirectional(LSTM(64, return_sequences=True, activation=config['activation'], kernel_regularizer=config['kernel_regularizer'], recurrent_dropout=config['recurrent_dropout'])),
    Dropout(config['dropout_rate']),
    BatchNormalization(),
    Bidirectional(LSTM(32, activation=config['activation'], kernel_regularizer=config['kernel_regularizer'], recurrent_dropout=config['recurrent_dropout'])),
    Dropout(config['dropout_rate']),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
    ])
  return model

# Initialize lists to store evaluation metrics for each fold
accuracy_list = []
f1_list = []
precision_list = []
recall_list = []
sensitivity_list = []
specificity_list = []
mcc_list = []
auc_list = []  # Initialize a list for AUC values


# Perform 5-fold cross-validation
skfold = StratifiedKFold(n_splits=5, shuffle = True, random_state = 5)
for train_index, val_index in skfold.split(X_train, y_train):
    # Split the data into training and validation sets for the current fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Ensure the model is reset for each fold
    model = get_model(config)
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

    # Train the model on the current fold
    history=model.fit(X_train_fold, y_train_fold, epochs=40, batch_size=16, verbose=0, validation_split=0.1, callbacks=[reduce_lr])


    # Plots the training and validation loss curves from Keras history
    # plt.plot(history.history['loss'], label='Train_Loss')
    # plt.plot(history.history['val_loss'], label='Validation_Loss')
    # plt.plot(history.history['accuracy'], label = 'Train_Accuracy')
    # plt.plot(history.history['val_accuracy'], label='Validation_Accuracy')
    # plt.title('Training and Validation')
    # plt.xlabel('Epoch')
    # plt.ylabel('Accuracy_Loss')
    # plt.legend()
    # plt.grid(True)  # Add grid lines for better readability
    # plt.show()

    # Evaluate the model on the validation set for the current fold
    y_val_pred = model.predict(X_val_fold)
    y_val_pred_binary = (y_val_pred > 0.5).astype(int)

    # Calculate confusion matrix for the current fold
    cm = confusion_matrix(y_val_fold, y_val_pred_binary)
    # Calculate AUC for the current fold
    auc = roc_auc_score(y_val_fold, y_val_pred)  # Use predicted probabilities for AUC calculation
    auc_list.append(auc)


    # Calculate performance evaluation metrics for the current fold
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]
    TP = cm[1, 1]

    # Accuracy
    accuracy = (TP + TN) / float(TP + TN + FP + FN)
    accuracy_list.append(accuracy)
    print("Fold Accuracy:", accuracy)

    # F1-Score
    f1 = 2 * TP / float(2 * TP + FP + FN)
    f1_list.append(f1)

    # Precision
    precision = TP / float(TP + FP)
    precision_list.append(precision)

    # Recall
    recall = TP / float(TP + FN)
    recall_list.append(recall)

    # Sensitivity
    sensitivity = TP / float(TP + FN)
    sensitivity_list.append(sensitivity)

    # Specificity
    specificity = TN / float(TN + FP)
    specificity_list.append(specificity)

    # MCC (Matthews Correlation Coefficient)
    mcc = ((TP * TN) - (FP * FN)) / float((np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))) or 1)
    mcc_list.append(mcc)

# Calculate average performance evaluation metrics across all folds
avg_accuracy = np.mean(accuracy_list)
avg_f1 = np.mean(f1_list)
avg_precision = np.mean(precision_list)
avg_recall = np.mean(recall_list)
avg_sensitivity = np.mean(sensitivity_list)
avg_specificity = np.mean(specificity_list)
avg_mcc = np.mean(mcc_list)

# Calculate average AUC across all folds
avg_auc = np.mean(auc_list)
# Print average performance evaluation metrics
print("Accuracy =", avg_accuracy)
print("F1 Score =", avg_f1)
print("Precision =", avg_precision)
print("Recall =", avg_recall)
print("Sensitivity =", avg_sensitivity)
print("Specificity =", avg_specificity)
print("MCC =", avg_mcc)
print("AUC =", avg_auc)




16/16 [==============================] - 3s 7ms/step
Fold Accuracy: 0.9587628865979382
16/16 [==============================] - 3s 7ms/step
Fold Accuracy: 0.9566115702479339
16/16 [==============================] - 2s 7ms/step
Fold Accuracy: 0.9524793388429752
16/16 [==============================] - 2s 7ms/step
Fold Accuracy: 0.9442148760330579
16/16 [==============================] - 3s 13ms/step
Fold Accuracy: 0.9566115702479339
Accuracy = 0.9537360483939679
F1 Score = 0.9503030783758917
Precision = 0.9542120585926138
Recall = 0.9470546957233636
Sensitivity = 0.9470546957233636
Specificity = 0.9596476940246734
MCC = 0.9075847574951782
AUC = 0.9872187595552342
